In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# reading input data
with open("../data/tinyshakespeare.txt", "r", encoding="utf-8") as file:
    data = file.read()

In [3]:
print(f"Length of dataset w.r.t chars: {len(data)}\n")
print(data[:1000])

Length of dataset w.r.t chars: 1115394

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger

In [4]:
# all unique chars
chars = sorted(list(set(data)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# tokenization (mapping for char to int)
ch_to_id = {ch:i for i, ch in enumerate(chars)}
id_to_ch = {i:ch for i, ch in enumerate(chars)}
encode = lambda str: [ch_to_id[ch] for ch in str] # tokenizer: takes string outputs lst on ints
decode = lambda lst: "".join([id_to_ch[i] for i in lst]) # decoder: takes lst of ids outputs a str

# tests
print(encode("Hello world!"))
print(decode(encode("Hello World!")))

[20, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]
Hello World!


In [6]:
# tokenizing enitire dataset and converting them into tensors
data = torch.tensor(encode(data), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # 1st 1000 char tensors

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
# train, val split
thresh = int(0.9 * len(data))
train_data = data[:thresh] # 90%
val_data = data[thresh:] # 10%

In [8]:
# dataloader: outs batches of x and y pairs
batch_size = 4 # n sequence to process parallaly
block_size = 8 # context length

def get_batches(split):
    """
    generate batched in and out pairs
    """
    data = train_data if split == "train" else val_data
    idx = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i + block_size] for i in idx])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in idx])
    return x, y

mini_x, mini_y = get_batches("train") # batch
print(f"Inputs shape: {mini_x.shape}\nInputs: {mini_x}\n")
print(f"Target shape: {mini_y.shape}\nTarget: {mini_y}\n")
print("--------------------------------------------------\n")

# insights into the batched data
for b in range(batch_size):
    for t in range(block_size):
        context = mini_x[b, :t+1]
        target = mini_y[b, t]
        print(f"Input: {context.tolist()}, Target: {target}")

Inputs shape: torch.Size([4, 8])
Inputs: tensor([[ 1, 58, 46, 43, 43,  6,  1, 59],
        [58, 46, 43, 43,  1, 61, 56, 53],
        [47, 52, 45,  1, 58, 46, 43, 43],
        [56, 60, 47, 52, 45, 51, 39, 52]])

Target shape: torch.Size([4, 8])
Target: tensor([[58, 46, 43, 43,  6,  1, 59, 54],
        [46, 43, 43,  1, 61, 56, 53, 52],
        [52, 45,  1, 58, 46, 43, 43,  1],
        [60, 47, 52, 45, 51, 39, 52, 10]])

--------------------------------------------------

Input: [1], Target: 58
Input: [1, 58], Target: 46
Input: [1, 58, 46], Target: 43
Input: [1, 58, 46, 43], Target: 43
Input: [1, 58, 46, 43, 43], Target: 6
Input: [1, 58, 46, 43, 43, 6], Target: 1
Input: [1, 58, 46, 43, 43, 6, 1], Target: 59
Input: [1, 58, 46, 43, 43, 6, 1, 59], Target: 54
Input: [58], Target: 46
Input: [58, 46], Target: 43
Input: [58, 46, 43], Target: 43
Input: [58, 46, 43, 43], Target: 1
Input: [58, 46, 43, 43, 1], Target: 61
Input: [58, 46, 43, 43, 1, 61], Target: 56
Input: [58, 46, 43, 43, 1, 61, 56], 

In [15]:
# bigram class
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers batch, time/context
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
bigram_model = BigramLanguageModel(vocab_size)
logits, loss = bigram_model(mini_x, mini_y)
print(f"Loss: {loss}")

Loss: 4.637217998504639


In [52]:
# predictions before training
print(decode(bigram_model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


,iCDQENe,!N-hi:EW
Es3Rj.eRylzQTmO
-$v&J3,tRDu;BZHskub:SvJD-lWEwDmvH.i.Mz:g,?;e:ILsWLM
wBW:F.TxsXlXfj


In [23]:
# optimizer
optimizer = torch.optim.AdamW(bigram_model.parameters(), lr=1e-3)

In [53]:
# Training bigram model
batch_size = 32
epochs = 10000

for epoch in range(epochs):
    # getting batches
    xb, yb = get_batches("train")
    
    # loss
    logits, loss = bigram_model(xb, yb)
    
    # setting grad to zero
    optimizer.zero_grad(set_to_none=True)
    
    # backprop
    loss.backward()
    
    # update params
    optimizer.step()
    if (epoch+1) % 1000 == 0:
        print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1000, Loss: 3.745992422103882
Epoch: 2000, Loss: 3.2971129417419434
Epoch: 3000, Loss: 2.6374945640563965
Epoch: 4000, Loss: 2.7319467067718506
Epoch: 5000, Loss: 2.515306234359741
Epoch: 6000, Loss: 2.4885129928588867
Epoch: 7000, Loss: 2.5165915489196777
Epoch: 8000, Loss: 2.4704906940460205
Epoch: 9000, Loss: 2.444457530975342
Epoch: 10000, Loss: 2.5025200843811035


In [61]:
# predictions after training
print(decode(bigram_model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))



Nont apellise, illaill a l l; trelyonu?
Bus
Whanghe wit d m.
An wil thin, b iss therad w sourd t r.
EDWe gheshas hieeshe nt cha soft oulfistint g,
me, breno hen jes: cky alers y thind wo a y toubeaplde s s?

A: st;
INaththuroul I SS:
NG ththoulllf-sol cke s m ya'e ororgrche tllmanousu am wal DUSI s t wir air y re ouncondeckil; ty; anfasthey wor wiceasen: sthabl, f s
Mas

AD y cer tol ath, seng.
Whs acoff's anghy besiamu :
Angel oved wher e o vern mero;
Sond bleey3NGofoferaves an hte ce ivacoure


In [65]:
# intuition for self-attention
B, T, C = 4, 8, 2 # Batch, Time(context length), Channels(embeddings or info at B,T)
x = torch.randn(B, T, C)
print(f"x Shape: {x.shape}")

# we want ith token to have attention to all tokens from i-T to i
# hence x[b, t] = mean of i<t x[b, i]
xbow = torch.zeros((B, T, C)) # averaging
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)
x[0], xbow[0]

x Shape: torch.Size([4, 8, 2])


(tensor([[ 1.7436,  0.4668],
         [ 0.3448,  0.2321],
         [ 0.0835, -0.2793],
         [ 0.1376,  0.0803],
         [ 0.1295,  1.4987],
         [ 0.2348,  0.7327],
         [-0.8297,  0.0313],
         [-0.4918, -1.2184]]),
 tensor([[1.7436, 0.4668],
         [1.0442, 0.3494],
         [0.7240, 0.1398],
         [0.5774, 0.1250],
         [0.4878, 0.3997],
         [0.4456, 0.4552],
         [0.2634, 0.3947],
         [0.1690, 0.1930]]))